# Limpeza de Dados do Dataset dos Resultados das Sprints

In [12]:
# Import das bibliotecas

import pandas as pd

In [13]:
# Lendo o arquivo csv
df = pd.read_csv('../data/sprint_results.csv', na_values=["\\N"])
df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,33,2,1.0,1,1,3,17,25:38.426,1538426.0,14.0,1:30.013,1
1,2,1061,1,131,44,1,2.0,2,2,2,17,+1.430,1539856.0,17.0,1:29.937,1
2,3,1061,822,131,77,3,3.0,3,3,1,17,+7.502,1545928.0,17.0,1:29.958,1
3,4,1061,844,6,16,4,4.0,4,4,0,17,+11.278,1549704.0,16.0,1:30.163,1
4,5,1061,846,1,4,6,5.0,5,5,0,17,+24.111,1562537.0,16.0,1:30.566,1


In [14]:
df.dtypes

resultId            int64
raceId              int64
driverId            int64
constructorId       int64
number              int64
grid                int64
position          float64
positionText       object
positionOrder       int64
points              int64
laps                int64
time               object
milliseconds      float64
fastestLap        float64
fastestLapTime     object
statusId            int64
dtype: object

As colunas numéricas `position`, `milliseconds` e `fastestLap` apresentam a mesma inconsistência, estão definidas como `float64`, mas representam valores numéricos inteiros (`int64`). Isso ocorre porque o tipo `int64` não suporta valores ausentes, com isso o seu tipo foi definido como `float64`.

In [15]:
# Corrigindo o tipo de dados das colunas para representar números inteiros
# O tipo 'Int64', com o 'I' maiúsculo, aceita valores ausentes e inteiros juntos
df['position'] = pd.to_numeric(df['position']).astype('Int64')
df['milliseconds'] = pd.to_numeric(df['milliseconds']).astype('Int64')
df['fastestLap'] = pd.to_numeric(df['fastestLap']).astype('Int64')
df.head()

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId
0,1,1061,830,9,33,2,1,1,1,3,17,25:38.426,1538426,14,1:30.013,1
1,2,1061,1,131,44,1,2,2,2,2,17,+1.430,1539856,17,1:29.937,1
2,3,1061,822,131,77,3,3,3,3,1,17,+7.502,1545928,17,1:29.958,1
3,4,1061,844,6,16,4,4,4,4,0,17,+11.278,1549704,16,1:30.163,1
4,5,1061,846,1,4,6,5,5,5,0,17,+24.111,1562537,16,1:30.566,1


In [16]:
# Função para converter o tempo, que está em string, em milisegundos
def time_to_milliseconds(t):
    if pd.isnull(t): # Caso seja um valor nulo, retorna nulo
        return None
    try: # Converte o tempo para milisegundos
        minutes, rest = t.split(':') # Separa os minutos do resto da string
        seconds, milliseconds = rest.split('.') # Separa os segundos e os milisegundos
        total_ms = (int(minutes) * 60 + int(seconds)) * 1000 + int(milliseconds) # Soma os minutos (convertido em milisegundos), segundos (convertido em milisegundos) e milisegundos
        return total_ms
    except ValueError: # Caso ocorra algum erro retorna nulo
        return None

# Cria a coluna 'q3_ms' com o tempo em milisegundos
df['fastestLapTime_ms'] = df['fastestLapTime'].apply(time_to_milliseconds).astype('Int64')
df.dtypes

resultId              int64
raceId                int64
driverId              int64
constructorId         int64
number                int64
grid                  int64
position              Int64
positionText         object
positionOrder         int64
points                int64
laps                  int64
time                 object
milliseconds          Int64
fastestLap            Int64
fastestLapTime       object
statusId              int64
fastestLapTime_ms     Int64
dtype: object

In [17]:
df.dtypes

resultId              int64
raceId                int64
driverId              int64
constructorId         int64
number                int64
grid                  int64
position              Int64
positionText         object
positionOrder         int64
points                int64
laps                  int64
time                 object
milliseconds          Int64
fastestLap            Int64
fastestLapTime       object
statusId              int64
fastestLapTime_ms     Int64
dtype: object

A inconsistência nos tipos das colunas foi corrigida.

## Identificando Dados Ausentes

In [18]:
# Verificando valores ausentes
df.isnull().sum()

resultId              0
raceId                0
driverId              0
constructorId         0
number                0
grid                  0
position             15
positionText          0
positionOrder         0
points                0
laps                  0
time                 20
milliseconds         20
fastestLap            9
fastestLapTime        9
statusId              0
fastestLapTime_ms     9
dtype: int64

Os campos ausentes nas colunas provavelmente foram causados pela não participação do piloto na corrida, indicadas pela coluna `positionText`, que registra códigos como "R" (retired), "F" (failed to qualify) ou "W" (withdrew). Essas ausências são, portanto, esperadas e coerentes com o contexto esportivo.

## Identificando Dados Duplicados

In [19]:
# Verificação de registros duplicados
df[df.duplicated(keep=False)]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId,fastestLapTime_ms


In [20]:
# Verificação de registros duplicados apenas na coluna 'resultId'
df[df.duplicated(subset='resultId' ,keep=False)]

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,fastestLapTime,statusId,fastestLapTime_ms


Não foi encontrado nenhum registro duplicado.

## Análise Descritiva das Variáveis Numéricas

In [21]:
df.describe()

,resultId,raceId,driverId,constructorId,number,grid,position,positionOrder,points,laps,milliseconds,fastestLap,statusId,fastestLapTime_ms
count,360.000000,360.000000,360.000000,360.000000,360.000000,360.000000,345.0,360.000000,360.000000,360.000000,340.0,351.0,360.000000,351.0
mean,180.500000,1106.944444,734.658333,94.366667,28.177778,9.922222,10.121739,10.500000,1.550000,19.583333,1806262.920588,10.638177,3.336111,86569.504274
std,104.067286,25.636134,276.412115,89.166421,23.796293,5.842363,5.59352,5.774307,2.496293,4.752803,154998.610339,6.267159,13.447963,14408.705039
min,1.000000,1061.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.000000,0.000000,0.000000,1498433.0,2.0,1.000000,68321.0
25%,90.750000,1084.000000,822.000000,6.000000,11.000000,5.000000,5.0,5.750000,0.000000,18.000000,1649334.0,5.0,1.000000,74144.5
50%,180.500000,1112.500000,840.000000,84.000000,22.000000,10.000000,10.0,10.500000,0.000000,19.000000,1842413.5,9.0,1.000000,84892.0
75%,270.250000,1126.000000,848.000000,210.000000,44.000000,15.000000,15.0,15.250000,3.000000,24.000000,1909140.5,17.0,1.000000,98790.5
max,360.000000,1143.000000,861.000000,215.000000,99.000000,20.000000,20.0,20.000000,8.000000,24.000000,2128165.0,24.0,130.000000,124760.0


Observando a análise estatística acima não foram identificadas inconsistências em nenhuma variável.

## Salvando o Dataset Tratado

In [22]:
df.to_csv('../data_cleaned/sprint_results.csv', index=False)